# 16. Natural Language Processing with RNNs and Attention

Looking at it from a certain perspective, the Turing test is an NLP task. This chapter will focus on how to tackle NLP tasks (albeit less complex than a Turing test) using RNNs. 

### Generating Shakespearean Text Using a Character RNN

Let's look at how to build a Char-RNN, a net that predicts the next character in a sentence. 

#### Creating the Training Dataset

Downloading the file from Andrej Karpathy's GitHub repo:

In [2]:
from tensorflow import keras

shakespeare_url = "https://github.com/karpathy/char-rnn" # shortcut URL
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

Exception: URL fetch failure on https://homl.info/shakespeare: None -- [Errno 11004] getaddrinfo failed